In [3]:
import os
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [10]:
df_l = pd.read_csv('data/listings.csv')

In [11]:
df_l.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,23163,https://www.airbnb.com/rooms/23163,20200229181007,2020-03-01,"Quiet 1BR directly in the Centre, Air-conditio...","Unique and elegant apartment rental in Prague,...",Unique and elegant apartment rental in Prague ...,"Unique and elegant apartment rental in Prague,...",none,Check my guidebook for this apartment to disco...,"To help you with your first steps in Prague, I...",The closest METRO station is called „Narodni t...,NaN,"Of course, you can contact me anytime during y...",Thank you for treating the property like it wa...,NaN,NaN,https://a0.muscache.com/im/pictures/01bbe32c-3...,NaN,5282,https://www.airbnb.com/users/show/5282,Klara,2008-12-17,"Prague, Prague, Czech Republic","Hello, \r\nglad to see that you are interested...",within an hour,97%,100%,t,https://a0.muscache.com/im/pictures/user/b7309...,https://a0.muscache.com/im/pictures/user/b7309...,Josefov,69.0,69.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,"Prague, Czech Republic, Czech Republic",Staré Město,Praha 1,NaN,Prague,Czech Republic,11000,Prague,"Prague, Czech Republic",CZ,Czech Republic,50.08229,14.41552,t,Apartment,Entire home/apt,4,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",775.0,"$3,717.00",NaN,NaN,"$2,546.00",NaN,2,$636.00,1,365,1,7,365,365,5.6,365.0,today,t,19,40,70,288,2020-03-01,19,3,2010-09-20,2019-07-27,98.0,9.0,10.0,10.0,10.0,10.0,10.0,f,NaN,NaN,t,f,moderate,f,f,69,68,0,0,0.17
1,23169,https://www.airbnb.com/rooms/23169,20200229181007,2020-03-01,M302-Cozy Serviced Studio in Center,Masna studio offers a lot of space and privacy...,* CENTRAL LOCATION * EQUIPPED KITCHEN * BALCON...,Masna studio offers a lot of space and privacy...,none,Check my guidebook for this apartment to disco...,"To help you with your first steps in Prague, I...",The closest METRO station is called „Staromest...,NaN,"Of course, you can contact me anytime during y...",Thank you for treating the property like it wa...,NaN,NaN,https://a0.muscache.com/im/pictures/35353e66-2...,NaN,5282,https://www.airbnb.com/users/show/5282,Klara,2008-12-17,"Prague, Prague, Czech Republic","Hello, \r\nglad to see that you are interested...",within an hour,97%,100%,t,https://a0.muscache.com/im/pictures/user/b7309...,https://a0.muscache.com/im/pictur

In [12]:
cols_to_drop = 'listing_url	scrape_id	last_scraped	name	summary	space	description	experiences_offered	neighborhood_overview	notes	transit	access	interaction	house_rules	thumbnail_url	medium_url	picture_url	xl_picture_url	host_url	host_name	host_location	host_about	host_thumbnail_url	host_picture_url	host_neighbourhood	host_listings_count	host_verifications	street	neighbourhood	city	state	zipcode	market	smart_location	country_code	country	latitude	longitude	is_location_exact	square_feet	weekly_price	monthly_price	maximum_nights	calendar_updated	has_availability	availability_30	availability_60	availability_90	availability_365	calendar_last_scraped	first_review	last_review	jurisdiction_names	require_guest_profile_picture	require_guest_phone_verification	calculated_host_listings_count'.split('\t')

In [13]:
df_lf = df_l.drop(cols_to_drop, axis = 1)

In [14]:
df_lf[['price', 'security_deposit', 'cleaning_fee', 'extra_people']].head()

,price,security_deposit,cleaning_fee,extra_people
0,"$3,717.00","$2,546.00",NaN,$636.00
1,"$2,493.00","$2,546.00",NaN,$636.00
2,"$2,540.00","$5,092.00",$458.00,$255.00
3,"$2,032.00","$3,819.00",$458.00,$204.00
4,"$1,039.00",$0.00,$350.00,$0.00


In [15]:
def convert_prices(df, cols):
    for col in cols:
        df[col] = df[col].str.replace('[$|,]', '').astype(float)
    return df

In [16]:
df_lf = convert_prices(df_lf, ['price', 'security_deposit', 'cleaning_fee', 'extra_people'])

In [17]:
df_a = pd.DataFrame(df_lf['amenities'].apply(lambda x: re.sub('[{|}|"]', '', x).split(',')))

In [18]:
df_a['id'] = df_lf['id']
df_a_con = pd.concat([pd.Series(row['id'], row['amenities']) for _, row in df_a.iterrows()]).reset_index()
df_a_con['val'] = 1
df_a_con.columns = ['amenity', 'id', 'value']

In [19]:
df_pivoted = df_a_con.pivot_table(index = 'id', columns = 'amenity', values = 'value', aggfunc='mean')

In [20]:
df_pivoted.head()

amenity,,toilet,24-hour check-in,Accessible-height bed,Accessible-height toilet,Air conditioning,Air purifier,Alfresco bathtub,Amazon Echo,BBQ grill,Baby bath,Baby monitor,Babysitter recommendations,Baking sheet,Balcony,Barbecue utensils,Bath towel,Bathroom essentials,Bathtub,Bathtub with bath chair,Beach essentials,Beach view,Beachfront,Bed linens,Bedroom comforts,Bidet,Body soap,Bread maker,Breakfast,Breakfast table,Building staff,Buzzer/wireless intercom,Cable TV,Carbon monoxide detector,Cat(s),Ceiling fan,Ceiling hoist,Central air conditioning,Changing table,Children’s books and toys,Children’s dinnerware,Cleaning before checkout,Coffee maker,Convection oven,Cooking basics,Crib,DVD player,Day bed,Disabled parking spot,Dishes and silverware,Dishwasher,Dog(s),Doorman,Double oven,Dryer,EV charger,Elevator,En suite bathroom,Espresso machine,Essentials,Ethernet connection,Extra pillows and blankets,Extra space around bed,Family/kid friendly,Fax machine,Fire extinguisher,Fire pit,Fireplace guards,Firm mattress,First aid kit,Fixed grab bars for shower,Fixed grab bars for toilet,Flat path to guest entrance,Formal dining area,Free parking on premises,Free street parking,Full kitchen,Game console,Garden or backyard,Gas oven,Gym,HBO GO,Hair dryer,Hammock,Handheld shower head,Hangers,Heat lamps,Heated floors,Heated towel rack,Heating,High chair,High-resolution computer monitor,Host greets you,Hot tub,Hot water,Hot water kettle,Indoor fireplace,Internet,Iron,Jetted tub,Keypad,Kitchen,Kitchenette,Lake access,Laptop friendly workspace,Lock on bedroom door,Lockbox,Long term stays allowed,Luggage dropoff allowed,Memory foam mattress,Microwave,Mini fridge,Mobile hoist,Mountain view,Mudroom,Murphy bed,Netflix,No stairs or steps to enter,Other,Other pet(s),Outdoor kitchen,Outdoor parking,Outdoor seating,Outlet covers,Oven,Pack ’n Play/travel crib,Paid parking off premises,Paid parking on premises,Patio or balcony,Pets allowed,Pets live on this property,Pillow-top mattress,Pocket wifi,Pool,Printer,Private entrance,Private hot tub,Private living room,Projector and screen,Rain shower,Refrigerator,Roll-in shower with chair,Room-darkening shades,Safety card,Sauna,Self check-in,Shampoo,Shared gym,Shared pool,Shower chair,Shower gel,Single level home,Ski-in/Ski-out,Smart TV,Smart lock,Smoke detector,Smoking allowed,Soaking tub,Sound system,Stair gates,Standing valet,Steam oven,Step-free shower,Stove,Suitable for events,Sun loungers,TV,Table corner guards,Terrace,Toilet paper,Trash can,Walk-in shower,Warming drawer,Washer,Waterfront,Well-lit path to entrance,Wheelchair accessible,Wide clearance to shower,Wide doorway to guest bathroom,Wide entrance,Wide entrance for guests,Wide entryway,Wide hallways,Wifi,Window guards,Wine cooler,translation missing: en.hosting_amenity_49,translation missing: en.hosting_amenity_50
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
23163,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
23169,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Na

In [21]:
df_pivoted_ratios = df_pivoted.isnull().sum() / df_pivoted.shape[0]

In [22]:
amenities_cols = df_pivoted_ratios[df_pivoted_ratios < 0.8].index.to_list()

In [23]:
amenities = df_pivoted[amenities_cols].fillna(0).reset_index()

In [24]:
amenities.head()

amenity,id,Bed linens,Carbon monoxide detector,Coffee maker,Cooking basics,Dishes and silverware,Dishwasher,Dryer,Elevator,Essentials,Extra pillows and blankets,Family/kid friendly,Fire extinguisher,First aid kit,Hair dryer,Hangers,Heating,Host greets you,Hot water,Internet,Iron,Kitchen,Laptop friendly workspace,Lock on bedroom door,Long term stays allowed,Luggage dropoff allowed,Microwave,Oven,Paid parking off premises,Pets allowed,Private entrance,Refrigerator,Shampoo,Smoke detector,Stove,TV,Washer,Wifi
0,23163,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,23169,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,26748,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,26755,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,27916,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0


In [25]:
df_all = df_lf.merge(amenities, on = 'id').drop('amenities', axis = 1)

In [26]:
features = df_all.isnull().sum() / df_all.shape[0]

In [27]:
df_all = df_all[features[features < 0.8].index.tolist()]

In [28]:
df_all.head()

,id,host_id,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,cancellation_policy,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,Bed linens,Carbon monoxide detector,Coffee maker,Cooking basics,Dishes and silverware,Dishwasher,Dryer,Elevator,Essentials,Extra pillows and blankets,Family/kid friendly,Fire extinguisher,First aid kit,Hair dryer,Hangers,Heating,Host greets you,Hot water,Internet,Iron,Kitchen,Laptop friendly workspace,Lock on bedroom door,Long term stays allowed,Luggage dropoff allowed,Microwave,Oven,Paid parking off premises,Pets allowed,Private entrance,Refrigerator,Shampoo,Smoke detector,Stove,TV,Washer,Wifi
0,23163,5282,2008-12-17,within an hour,97%,100%,t,69.0,t,t,Praha 1,Apartment,Entire home/apt,4,1.0,1.0,2.0,Real Bed,3717.0,2546.0,NaN,2,636.0,1,1,7,365,365,5.6,365.0,19,3,98.0,9.0,10.0,10.0,10.0,10.0,10.0,f,t,f,moderate,68,0,0,0.17,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,23169,5282,2008-12-17,within an hour,97%,100%,t,69.0,t,t,Praha 1,Apartment,Entire home/apt,4,1.0,1.0,2.0,Real Bed,2493.0,2546.0,NaN,2,636.0,1,1,7,365,365,1.2,365.0,113,1,95.0,10.0,10.0,10.0,10.0,10.0,9.0,f,t,f,moderate,68,0,0,0.95,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,26748,113902,2010-04-26,within an hour,100%,100%,t,3.0,t,t,Praha 7,Apartment,Entire home/apt,6,1.0,2.0,2.0,Real Bed,2540.0,5092.0,458.0,4,255.0,3,2,7,1125,1125,3.2,1125.0,345,54,97.0,10.0,10.0,10.0,10.0,10.0,10.0,f,t,f,moderate,3,0,0,3.00,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,26755,113902,2010-04-26,within an hour,100%,100%,t,3.0,t,t,Praha 1,Apartment,Entire home/apt,4,1.5,1.0,2.0,Real Bed,2032.0,3819.0,458.0,3,204.0,3,2,7,1125,1125,3.2,1125.0,246,53,99.0,10.0,10.0,10.0,10.0,10.0,10.0,f,t,f,moderate,3,0,0,4.22,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,27916,120384,2010-05-08,within an hour,100%,100%,t,3.0,t,t,Praha 3,Condominium,Entire home/apt,2,1.0,0.0,1.0,Real Bed,1039.0,0.0,350.0,1,0.0,2,2,2,50,50,2.0,50.0,411,48,96.0,10.0,10.0,10.0,10.0,10.0,10.0,f,t,f,moderate,3,0,0,3.52,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0


In [29]:
df_all['host_response_rate'] = df_all['host_response_rate'].str.replace('%', '').astype(float)

In [30]:
df_all['host_since'] = df_all['host_since'].apply(pd.to_datetime)

In [31]:
for col in df_all.dtypes[df_all.dtypes == 'float'].index.tolist():
    df_all[col] = df_all[col].fillna(df_all[col].mean())

In [32]:
list(map(lambda x: df_all[x].unique(), df_all.columns.tolist()))

[array([   23163,    23169,    26748, ..., 42617334, 42617339, 42618028],
       dtype=int64),
 array([     5282,    113902,    120384, ..., 339331837,  65456966,
        339478901], dtype=int64),
 array(['2008-12-17T00:00:00.000000000', '2010-04-26T00:00:00.000000000',
        '2010-05-08T00:00:00.000000000', ...,
        '2020-02-27T00:00:00.000000000', '2020-02-28T00:00:00.000000000',
        '2020-02-29T00:00:00.000000000'], dtype='datetime64[ns]'),
 array(['within an hour', 'within a few hours', 'within a day', nan,
        'a few days or more'], dtype=object),
 array([ 97.        , 100.        ,  67.        ,  99.        ,
         94.        ,  95.        ,  96.37769328,  93.        ,
         88.        ,  33.        ,  80.        ,  92.        ,
         50.        ,  90.        ,  17.        ,  96.        ,
          0.        ,  75.        ,  98.        ,  70.        ,
         60.        ,  30.        ,  83.        ,  82.        ,
         86.        ,  25.        ,  89.   

In [34]:
df_all.groupby('neighbourhood_cleansed')['neighbourhood_cleansed'].count() / df_all.shape[0]

neighbourhood_cleansed
Benice             0.000343
Běchovice          0.000069
Březiněves         0.000412
Dolní Chabry       0.001030
Dolní Měcholupy    0.000275
Dolní Počernice    0.000618
Klánovice          0.000824
Koloděje           0.000206
Kolovraty          0.000206
Královice          0.000069
Kunratice          0.001442
Libuš              0.001030
Lipence            0.000206
Lysolaje           0.000206
Nebušice           0.000412
Petrovice          0.000893
Praha 1            0.283929
Praha 10           0.052747
Praha 11           0.005220
Praha 12           0.003503
Praha 13           0.007555
Praha 14           0.003915
Praha 15           0.003228
Praha 16           0.001030
Praha 17           0.002816
Praha 18           0.002060
Praha 19           0.000275
Praha 2            0.166690
Praha 20           0.001030
Praha 21           0.000618
Praha 22           0.001099
Praha 3            0.133310
Praha 4            0.043269
Praha 5            0.087706
Praha 6            0.0467